# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [505]:
import numpy as np

In [506]:
def get_threshold(feature, y):
    best_thr = None
    # YOUR CODE HERE
    indexes = np.argsort(feature)
    feature = feature[indexes]
    y = y[indexes]
    y_sum_right = np.sum(y)
    y_sum_squared_right = np.sum(y**2)
    y_sum_left = 0
    y_sum_squared_left = 0
    N_right = y.shape[0]
    N_left = 0
    thr = 0
    best_thr = y[0]
    Q_0 = 324523451351351351351
    j = 0
    for i in y:
        y_sum_right -= i
        y_sum_squared_right -= i**2
        y_sum_left += i
        y_sum_squared_left += i**2
        N_right -= 1
        N_left += 1
        j += 1
        if N_right == 0:
            continue
        if N_right != 0:
            Q = (y_sum_squared_left - (y_sum_left ** 2) / N_left + y_sum_squared_right - (y_sum_right ** 2) / N_right)
        if Q < Q_0:
            left_mean = np.mean(y[:j])
            right_mean = np.mean(y[j:])
            Q_0 = Q
            best_thr = feature[j - 1]
    return Q_0, best_thr, left_mean, right_mean


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    print(X.shape)
    for j in range(X.shape[1]):
        f_score, f_thr, left_mean, right_mean = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
            l = left_mean
            r = right_mean
    return best_f, best_thr, l, r

In [513]:
def predicter(features, value, left, right, j):
    y_predict = []
    for i in features:
        i = i[j]
        if i <= value:
            y_predict.append(left)
        else:
            y_predict.append(right)
    return np.asarray(y_predict)

In [514]:
print(mean_squared_error(y, predicter(X,  6.939, 19.933720930232557, 37.238157894736844, 5)))

46.19909167710848


In [515]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [516]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))


0.0
46.19909167710848


In [517]:
train_stump(X, y)

(506, 13)


(5, 6.939, 19.933720930232557, 37.238157894736844)

In [518]:
print(mean_squared_error(y, predicter(X,  6.939, 19.933720930232557, 37.238157894736844, 5)))

46.19909167710848
